In [1]:
from proj1_helpers import *
import matplotlib.pyplot as plt
import numpy as np
from implementation import *
from Visualization import *
from data_preproc import preprocessing
from features_func import generate_features
from Learning import learning
from crossval import *

In [2]:
train_path = 'data/train.csv'
y, tX, ids = load_csv_data(train_path, sub_sample=False)
nSample_tr, nFeature_tr = tX.shape

test_path = 'data/test.csv'
_, tX_te, ids_te = load_csv_data(test_path, sub_sample=False)
y_te = np.zeros((tX_te.shape[0],))
nSample_te, nFeature_te = tX.shape

complete_tX = np.vstack((tX,tX_te))
complete_y = np.append(y,y_te)
complete_ids = np.append(ids,ids_te)

In [3]:
model0, model1, model2 = preprocessing(complete_tX,complete_y,complete_ids)

all_model = [model0, model1, model2]

In [4]:
print(model1['tX_te'].shape)

(175338, 18)


In [5]:
best_param = [3.5938136638046256e-05,3.5938136638046256e-05,0.0001]
degree = 2

y_final = []
ids_final = []

f_scores = []

for i, model_i in enumerate(all_model):
    print(model_i['tX_tr'].shape)
    print(model_i['y_tr'].shape)
    print(model_i['tX_te'].shape)
    print(model_i['te_id'].shape)

    scores = []
    nSample, nFeature = model_i['tX_tr'].shape

    print('ridge regression')
    tX_newfeat = generate_features(model_i['tX_tr'], degree)
    w,_ = ridge_regression(model_i['y_tr'], tX_newfeat, best_param[i])
    
    pred = predict_labels(w, generate_features(model_i['tX_te'], degree))
    ids_final = np.append(ids_final, model_i['te_id'])
    y_final = np.append(y_final, pred)
    print('ids_final shape')
    print(ids_final.shape)
    print('y_final shape')
    print(y_final.shape)
    
    
    seed = 1
    k_fold = 4

    # split data in k fold
    k_indices = build_k_indices(model_i['y_tr'], k_fold, seed)
    for k in range(k_fold):
        _,_,score = cross_validation(model_i['y_tr'], tX_newfeat, k_indices, k, best_param[i])
        score *= 100
        f_scores.append(score)
        

    
    
f_scores

(99913, 16)
(99913,)
(227458, 16)
(227458,)
ridge regression
ids_final shape
(227458,)
y_final shape
(227458,)
Ridge inside cross val started
Ridge inside cross val started
Ridge inside cross val started
Ridge inside cross val started
(77544, 18)
(77544,)
(175338, 18)
(175338,)
ridge regression
ids_final shape
(402796,)
y_final shape
(402796,)
Ridge inside cross val started
Ridge inside cross val started
Ridge inside cross val started
Ridge inside cross val started
(72543, 24)
(72543,)
(165442, 24)
(165442,)
ridge regression
ids_final shape
(568238,)
y_final shape
(568238,)
Ridge inside cross val started
Ridge inside cross val started
Ridge inside cross val started
Ridge inside cross val started


[array([84.21410842]),
 array([84.23412603]),
 array([84.91072143]),
 array([85.15493634]),
 array([79.94944806]),
 array([80.2537914]),
 array([80.8057361]),
 array([80.06293201]),
 array([82.76261373]),
 array([82.61924455]),
 array([82.77915633]),
 array([82.99972429])]

f_scores represent the accuracy for each pass

In [6]:
print(y_final.shape)

(568238,)


In [7]:
create_csv_submission(ids_final, y_final, "final_submission.csv")